In [ ]:
import os
import re
import cv2
import easyocr
from google.colab import drive
drive.mount('/content/drive')
INPUT_DIR = '/content/drive/My Drive/img'
PHONE_RE = re.compile(r'0[67]\d{8}')
reader = easyocr.Reader(['fr'], gpu=False)

def preprocess(img):
    """
    Optionnel : améliorer contraste / réduire bruit avant OCR.
    Ici on convertit en LAB + CLAHE sur le L pour
    booster un peu le contraste localement.
    """
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l2 = clahe.apply(l)
    lab2 = cv2.merge((l2,a,b))
    return cv2.cvtColor(lab2, cv2.COLOR_LAB2BGR)

def extract_phone_numbers(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return []
    img = preprocess(img)
    results = reader.readtext(img)
    found = []
    for bbox, text, prob in results:
        txt = re.sub(r'[^0-9]', '', text)
        for m in PHONE_RE.findall(txt):
            found.append((m, prob))
    best = {}
    for num, conf in found:
        if num not in best or conf > best[num]:
            best[num] = conf
    return sorted(best.keys(), key=lambda x: best[x], reverse=True)

def main():
    for fname in sorted(os.listdir(INPUT_DIR)):
        if not fname.lower().endswith(('.jpg','.jpeg','.png','.bmp','.tif')):
            continue
        path = os.path.join(INPUT_DIR, fname)
        nums = extract_phone_numbers(path)
        print(f"{fname} → {nums or ['<aucun>']}")

if __name__ == "__main__":
    main()

Mounted at /content/drive
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteWhatsApp Image 2025-07-03 at 12.11.59 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.00 PM (2).jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.00 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.01 PM.jpeg → ['0707639242']
WhatsApp Image 2025-07-03 at 12.12.05 PM (2).jpeg → ['0699347289']
WhatsApp Image 2025-07-03 at 12.12.05 PM (4).jpeg → ['0726168484']
WhatsApp Image 2025-07-03 at 12.12.05 PM.jpeg → ['0630424085']
WhatsApp Image 2025-07-03 at 12.12.06 PM (3).jpeg → ['0601112531']
WhatsApp Image 2025-07-03 at 12.12.06 PM (5).jpeg → ['0680469179']
WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg → ['0698455356']
WhatsApp Image 2025-07-03 at 12.12.07 PM (4).jpeg → ['0722244482']
WhatsApp Image 2025-07-03 at 12.12.08 PM.jpeg → ['0608131898']
WhatsApp Image 2025-07-03 at 12.12.09 PM.jpeg → ['0711130690']
WhatsApp Image 2025-07-03 at 12.12.10 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-0

In [ ]:
import os
import re
import cv2
import numpy as np
import easyocr
from google.colab import drive
drive.mount('/content/drive')
INPUT_DIR = '/content/drive/My Drive/img'

PHONE_RE = re.compile(r'(?<![\w])0[67][0-9OIl]{8}')
reader = easyocr.Reader(['fr'], gpu=True)

def is_blue_or_white(img_crop):
    hsv = cv2.cvtColor(img_crop, cv2.COLOR_BGR2HSV)
    mask_blue = cv2.inRange(hsv, (90, 50, 50), (130, 255, 255))
    mask_white = cv2.inRange(hsv, (0, 0, 180), (180, 60, 255))

    blue_ratio = cv2.countNonZero(mask_blue) / (img_crop.shape[0] * img_crop.shape[1])
    white_ratio = cv2.countNonZero(mask_white) / (img_crop.shape[0] * img_crop.shape[1])

    return blue_ratio > 0.2 or white_ratio > 0.2

def preprocess(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl1 = clahe.apply(gray)
    _, thresh = cv2.threshold(cl1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)

def extract_phone_numbers(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return []

    img = preprocess(img)
    results = reader.readtext(img)

    found = []
    for bbox, text, prob in results:
        pts = [tuple(map(int, p)) for p in bbox]
        x_min = max(min(p[0] for p in pts), 0)
        x_max = max(p[0] for p in pts)
        y_min = max(min(p[1] for p in pts), 0)
        y_max = max(p[1] for p in pts)

        roi = img[y_min:y_max, x_min:x_max]
        if roi.size == 0 or not is_blue_or_white(roi):
            continue
        cleaned = text.upper().replace('O', '0').replace('I', '1').replace('l', '1')
        digits = re.sub(r'[^0-9]', '', cleaned)

        for m in PHONE_RE.findall(digits):
            if len(m) == 10:
                found.append((m, prob))
    best = {}
    for num, conf in found:
        if num not in best or conf > best[num]:
            best[num] = conf

    return sorted(best.keys(), key=lambda x: best[x], reverse=True)

def main():
    total = 0
    detectees = 0

    for fname in sorted(os.listdir(INPUT_DIR)):
        if not fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif')):
            continue

        total += 1
        path = os.path.join(INPUT_DIR, fname)
        nums = extract_phone_numbers(path)

        if nums:
            detectees += 1

        print(f"{fname} → {nums or ['<aucun>']}")

    if total > 0:
        pourcentage = (detectees / total) * 100
        print(f"\n📊 Taux de détection : {detectees}/{total} images ({pourcentage:.2f}%)")
    else:
        print("❌ Aucun fichier image trouvé.")

if __name__ == "__main__":
    main()


Mounted at /content/drive
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteWhatsApp Image 2025-07-03 at 12.11.59 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.00 PM (2).jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.00 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.01 PM.jpeg → ['0707639242']
WhatsApp Image 2025-07-03 at 12.12.05 PM (2).jpeg → ['0699347289']
WhatsApp Image 2025-07-03 at 12.12.05 PM (4).jpeg → ['0726168484']
WhatsApp Image 2025-07-03 at 12.12.05 PM.jpeg → ['0630424085']
WhatsApp Image 2025-07-03 at 12.12.06 PM (3).jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.06 PM (5).jpeg → ['0680469179']
WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg → ['0698455356']
WhatsApp Image 2025-07-03 at 12.12.07 PM (4).jpeg → ['0722244482']
WhatsApp Image 2025-07-03 at 12.12.08 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.09 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.10 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.1

In [ ]:
import os
import re
import cv2
import easyocr
INPUT_DIR = '/content/drive/My Drive/img'
PHONE_RE = re.compile(r'(?<![\w])0[67]\d{8}')
reader = easyocr.Reader(['fr'], gpu=True)
def preprocess(img):
    """
    Améliore le contraste localement avec CLAHE (sur le canal L en LAB).
    """
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l2 = clahe.apply(l)
    lab2 = cv2.merge((l2, a, b))
    return cv2.cvtColor(lab2, cv2.COLOR_LAB2BGR)

def extract_phone_numbers(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return []
    img = preprocess(img)
    results = reader.readtext(img)
    found = []
    for bbox, text, prob in results:
        text_cleaned = text.upper().replace('O', '0').replace('I', '1').replace('L', '1')
        digits = re.sub(r'[^0-9]', '', text_cleaned) # Assign the result to digits
        if len(digits) > 11:
          continue

        for m in PHONE_RE.findall(digits):
            if len(m) == 10:
                found.append((m, prob))
    best = {}
    for num, conf in found:
        if num not in best or conf > best[num]:
            best[num] = conf
    return sorted(best.keys(), key=lambda x: best[x], reverse=True)

def main():
    total = 0
    detectees = 0

    for fname in sorted(os.listdir(INPUT_DIR)):
        if not fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif')):
            continue
        total += 1
        path = os.path.join(INPUT_DIR, fname)
        nums = extract_phone_numbers(path)

        if nums:
            detectees += 1

        print(f"{fname} → {nums or ['<aucun>']}")


    if total > 0:
        pourcentage = (detectees / total) * 100
        print(f"\n📊 Taux de détection : {detectees}/{total} images ({pourcentage:.2f}%)")
    else:
        print("❌ Aucun fichier image trouvé.")

if __name__ == "__main__":
    main()

WhatsApp Image 2025-07-03 at 12.11.59 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.00 PM (2).jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.00 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.01 PM.jpeg → ['0707639242']
WhatsApp Image 2025-07-03 at 12.12.05 PM (2).jpeg → ['0699347289']
WhatsApp Image 2025-07-03 at 12.12.05 PM (4).jpeg → ['0726168484']
WhatsApp Image 2025-07-03 at 12.12.05 PM.jpeg → ['0630424085']
WhatsApp Image 2025-07-03 at 12.12.06 PM (3).jpeg → ['0601112531']
WhatsApp Image 2025-07-03 at 12.12.06 PM (5).jpeg → ['0680469179']
WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg → ['0698455356']
WhatsApp Image 2025-07-03 at 12.12.07 PM (4).jpeg → ['0722244482']
WhatsApp Image 2025-07-03 at 12.12.08 PM.jpeg → ['0608131898']
WhatsApp Image 2025-07-03 at 12.12.09 PM.jpeg → ['0711130690']
WhatsApp Image 2025-07-03 at 12.12.10 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.11 PM (2).jpeg → ['0726254301']
WhatsApp Image 2025-07-03 at 12.12.

In [ ]:
import os
import re
import cv2
import easyocr

INPUT_DIR = '/content/drive/My Drive/img'
PHONE_RE = re.compile(r'0[67][0-9OIlL]{8}')
reader = easyocr.Reader(['fr'], gpu=True)

def preprocess(img):
    """
    Prétraitement : agrandissement, débruitage, amélioration du contraste.
    """
    img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    return cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)

def extract_phone_numbers(image_path):
    """
    Extrait les numéros de téléphone à partir d'une image.
    """
    img = cv2.imread(image_path)
    if img is None:
        return []

    img = preprocess(img)
    results = reader.readtext(img, detail=1, paragraph=False)
    found = []

    for result in results:
        if len(result) == 3:
            _, text, prob = result
        elif len(result) == 2:
            text, prob = result
        else:
            continue

        text_cleaned = text.upper().replace('O', '0').replace('I', '1').replace('L', '1')
        digits = re.sub(r'[^0-9]', '', text_cleaned)

        if len(digits) > 11:
            continue

        if len(digits) == 10 and digits.startswith(('06', '07')):
            found.append((digits, prob))

        for m in PHONE_RE.findall(digits):
            clean = re.sub(r'[^\d]', '', m)
            if len(clean) == 10:
                found.append((clean, prob))

    best = {}
    for num, conf in found:
        if num not in best or conf > best[num]:
            best[num] = conf

    return sorted(best.keys(), key=lambda x: best[x], reverse=True)

def main():
    total = 0
    detectees = 0

    for fname in sorted(os.listdir(INPUT_DIR)):
        if not fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif')):
            continue

        total += 1
        path = os.path.join(INPUT_DIR, fname)
        nums = extract_phone_numbers(path)

        if nums:
            detectees += 1

        print(f"{fname} → {nums or ['<aucun>']}")

    if total > 0:
        pourcentage = (detectees / total) * 100
        print(f"\n📊 Taux de détection : {detectees}/{total} images ({pourcentage:.2f}%)")
    else:
        print("❌ Aucun fichier image trouvé.")

if __name__ == "__main__":
    main()


WhatsApp Image 2025-07-03 at 12.11.59 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.00 PM (2).jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.00 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.01 PM.jpeg → ['0707639242']
WhatsApp Image 2025-07-03 at 12.12.05 PM (2).jpeg → ['0699347289']
WhatsApp Image 2025-07-03 at 12.12.05 PM (4).jpeg → ['0726168484']
WhatsApp Image 2025-07-03 at 12.12.05 PM.jpeg → ['0630424085']
WhatsApp Image 2025-07-03 at 12.12.06 PM (3).jpeg → ['0601112531']
WhatsApp Image 2025-07-03 at 12.12.06 PM (5).jpeg → ['0680469179']
WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg → ['0698455356']
WhatsApp Image 2025-07-03 at 12.12.07 PM (4).jpeg → ['0722244482']
WhatsApp Image 2025-07-03 at 12.12.08 PM.jpeg → ['0608131898']
WhatsApp Image 2025-07-03 at 12.12.09 PM.jpeg → ['0711130690']
WhatsApp Image 2025-07-03 at 12.12.10 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.11 PM (2).jpeg → ['0726254301']
WhatsApp Image 2025-07-03 at 12.12.

In [ ]:
import os
import re
import cv2
import easyocr
INPUT_DIR = '/content/drive/My Drive/img'
PHONE_RE = re.compile(r'0[67][0-9OIlL]{8}')
reader = easyocr.Reader(['fr'], gpu=True)

def preprocess_basic(img):
    """Prétraitement standard (bon pour texte imprimé propre)."""
    img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    return cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)

def preprocess_handwritten(img):
    """Prétraitement plus agressif (bon pour texte manuscrit)."""
    img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 9, 75, 75)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    gray = clahe.apply(gray)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 31, 10)
    return cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)

def clean_text(text):
    """Corrige erreurs OCR classiques."""
    return text.upper().replace('O', '0').replace('I', '1').replace('L', '1')

def extract_from_img(img):
    """OCR sur une image donnée (prétraitée)."""
    results = reader.readtext(img, detail=1, paragraph=False)
    found = []

    for result in results:
        if len(result) == 3:
            _, text, prob = result
        elif len(result) == 2:
            text, prob = result
        else:
            continue

        text_cleaned = clean_text(text)
        digits = re.sub(r'[^0-9]', '', text_cleaned)

        if len(digits) > 11 or len(digits) < 8:
            continue

        if len(digits) == 10 and digits.startswith(('06', '07')):
            found.append((digits, prob))

        for m in PHONE_RE.findall(digits):
            clean = re.sub(r'[^\d]', '', m)
            if len(clean) == 10:
                found.append((clean, prob))

    return found

def extract_phone_numbers(image_path):
    """Essaye plusieurs méthodes de prétraitement et sélectionne le meilleur résultat."""
    img = cv2.imread(image_path)
    if img is None:
        return []

    found_all = []
    for preprocess_fn in [preprocess_basic, preprocess_handwritten]:
        processed = preprocess_fn(img)
        found_all.extend(extract_from_img(processed))

    best = {}
    for num, conf in found_all:
        if num not in best or conf > best[num]:
            best[num] = conf

    return sorted(best.keys(), key=lambda x: best[x], reverse=True)

def main():
    total = 0
    detectees = 0

    for fname in sorted(os.listdir(INPUT_DIR)):
        if not fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif')):
            continue

        total += 1
        path = os.path.join(INPUT_DIR, fname)
        nums = extract_phone_numbers(path)

        if nums:
            detectees += 1

        print(f"{fname} → {nums or ['<aucun>']}")

    if total > 0:
        pourcentage = (detectees / total) * 100
        print(f"\n📊 Taux de détection : {detectees}/{total} images ({pourcentage:.2f}%)")
    else:
        print("❌ Aucun fichier image trouvé.")

if __name__ == "__main__":
    main()


WhatsApp Image 2025-07-03 at 12.11.59 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.00 PM (2).jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.00 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.01 PM.jpeg → ['0707639242']
WhatsApp Image 2025-07-03 at 12.12.05 PM (2).jpeg → ['0699347289']
WhatsApp Image 2025-07-03 at 12.12.05 PM (4).jpeg → ['0726168484']
WhatsApp Image 2025-07-03 at 12.12.05 PM.jpeg → ['0630424085']
WhatsApp Image 2025-07-03 at 12.12.06 PM (3).jpeg → ['0601112531']
WhatsApp Image 2025-07-03 at 12.12.06 PM (5).jpeg → ['0680469179']
WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg → ['0698455356']
WhatsApp Image 2025-07-03 at 12.12.07 PM (4).jpeg → ['0722244482']
WhatsApp Image 2025-07-03 at 12.12.08 PM.jpeg → ['0608131898']
WhatsApp Image 2025-07-03 at 12.12.09 PM.jpeg → ['0711130690']
WhatsApp Image 2025-07-03 at 12.12.10 PM.jpeg → ['<aucun>']
WhatsApp Image 2025-07-03 at 12.12.11 PM (2).jpeg → ['0726254301', '0736035177']
WhatsApp Image 2025-0

In [ ]:
import os
import re
import cv2
import easyocr

INPUT_DIR = '/content/drive/My Drive/img'

reader = easyocr.Reader(['fr'], gpu=True)
PHONE_RE = re.compile(r'0[67][0-9OIlL]{8}')

def clean_text(text):
    return text.upper().replace('O', '0').replace('I', '1').replace('L', '1')

def preprocess_basic(img):
    img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    return cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)

def preprocess_handwritten(img):
    img = cv2.resize(img, None, fx=2.5, fy=2.5, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    enhanced = cv2.addWeighted(gray, 1.5, blurred, -0.5, 0)
    thresh = cv2.adaptiveThreshold(enhanced, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY_INV, 31, 15)
    return cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)

def extract_from_img(img):
    results = reader.readtext(img, detail=1, paragraph=False)
    found = []

    for result in results:
        if len(result) == 3:
            _, text, conf = result
        else:
            continue

        cleaned = clean_text(text)
        digits = re.sub(r'[^0-9]', '', cleaned)

        # Ne garder que les vrais numéros valides
        if len(digits) == 10 and digits.startswith(('06', '07')):
            found.append((digits, conf))
        else:

            for match in PHONE_RE.findall(digits):
                num = re.sub(r'[^0-9]', '', match)
                if len(num) == 10:
                    found.append((num, conf))

    return found

def extract_phone_number(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None

    candidates = []

    for preprocess_fn in [preprocess_basic, preprocess_handwritten]:
        processed = preprocess_fn(img)
        candidates += extract_from_img(processed)

    if candidates:
        best = max(candidates, key=lambda x: x[1])
        return best[0]
    else:
        return None

def main():
    total = 0
    detectees = 0

    for fname in sorted(os.listdir(INPUT_DIR)):
        if not fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif')):
            continue

        total += 1
        path = os.path.join(INPUT_DIR, fname)
        num = extract_phone_number(path)

        if num:
            detectees += 1
            print(f"{fname} → ✅ {num}")
        else:
            print(f"{fname} → ❌ Aucun numéro détecté")

    print(f"\n📊 Résultat final : {detectees}/{total} images détectées ({(detectees/total)*100:.2f}%)")

if __name__ == "__main__":
    main()


WhatsApp Image 2025-07-03 at 12.11.59 PM.jpeg → ❌ Aucun numéro détecté
WhatsApp Image 2025-07-03 at 12.12.00 PM (2).jpeg → ❌ Aucun numéro détecté
WhatsApp Image 2025-07-03 at 12.12.00 PM.jpeg → ❌ Aucun numéro détecté
WhatsApp Image 2025-07-03 at 12.12.01 PM.jpeg → ✅ 0707639242
WhatsApp Image 2025-07-03 at 12.12.05 PM (2).jpeg → ✅ 0699347289
WhatsApp Image 2025-07-03 at 12.12.05 PM (4).jpeg → ✅ 0726168484
WhatsApp Image 2025-07-03 at 12.12.05 PM.jpeg → ✅ 0630424085
WhatsApp Image 2025-07-03 at 12.12.06 PM (3).jpeg → ✅ 0601112531
WhatsApp Image 2025-07-03 at 12.12.06 PM (5).jpeg → ✅ 0680469179
WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg → ✅ 0698455356
WhatsApp Image 2025-07-03 at 12.12.07 PM (4).jpeg → ✅ 0722244482
WhatsApp Image 2025-07-03 at 12.12.08 PM.jpeg → ✅ 0608131898
WhatsApp Image 2025-07-03 at 12.12.09 PM.jpeg → ✅ 0711130690
WhatsApp Image 2025-07-03 at 12.12.10 PM.jpeg → ❌ Aucun numéro détecté
WhatsApp Image 2025-07-03 at 12.12.11 PM (2).jpeg → ✅ 0726254301
WhatsApp Imag

In [ ]:
!git clone https://github.com/ultralytics/yolov5



Cloning into 'yolov5'...
remote: Enumerating objects: 17521, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 17521 (delta 9), reused 0 (delta 0), pack-reused 17497 (from 4)
Receiving objects: 100% (17521/17521), 16.61 MiB | 18.13 MiB/s, done.
Resolving deltas: 100% (11998/11998), done.


In [ ]:
%cd yolov5
!pip install -r requirements.txt

/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82


In [ ]:
from google.colab import files
files.upload()


!unzip phone_number_detector.v1i.yolov5pytorch.zip -d /content/dataset/



Saving phone_number_detector.v1i.yolov5pytorch.zip to phone_number_detector.v1i.yolov5pytorch.zip
Archive:  phone_number_detector.v1i.yolov5pytorch.zip
  inflating: /content/dataset/README.dataset.txt  
  inflating: /content/dataset/README.roboflow.txt  
  inflating: /content/dataset/data.yaml  
   creating: /content/dataset/train/
   creating: /content/dataset/train/images/
 extracting: /content/dataset/train/images/WhatsApp-Image-2025-07-03-at-12_11_59-PM_jpeg.rf.052d1d0629688287c0aa20ebce46efb9.jpg  
 extracting: /content/dataset/train/images/WhatsApp-Image-2025-07-03-at-12_12_00-PM-2-_jpeg.rf.b305c356e16e9fb3210c912ebb5440a0.jpg  
 extracting: /content/dataset/train/images/WhatsApp-Image-2025-07-03-at-12_12_00-PM_jpeg.rf.82065192facd2e343ddc893cec101f47.jpg  
 extracting: /content/dataset/train/images/WhatsApp-Image-2025-07-03-at-12_12_01-PM_jpeg.rf.24b4bd9afada254e36e52edb98efa2be.jpg  
 extracting: /content/dataset/train/images/WhatsApp-Image-2025-07-03-at-12_12_05-PM-2-_jpeg.rf.

In [ ]:
!ls -R /content/dataset/

/content/dataset/:
data.yaml  README.dataset.txt  README.roboflow.txt  train

/content/dataset/train:
images	labels

/content/dataset/train/images:
WhatsApp-Image-2025-07-03-at-12_11_59-PM_jpeg.rf.052d1d0629688287c0aa20ebce46efb9.jpg
WhatsApp-Image-2025-07-03-at-12_12_00-PM-2-_jpeg.rf.b305c356e16e9fb3210c912ebb5440a0.jpg
WhatsApp-Image-2025-07-03-at-12_12_00-PM_jpeg.rf.82065192facd2e343ddc893cec101f47.jpg
WhatsApp-Image-2025-07-03-at-12_12_01-PM_jpeg.rf.24b4bd9afada254e36e52edb98efa2be.jpg
WhatsApp-Image-2025-07-03-at-12_12_05-PM-2-_jpeg.rf.1d67ac77dbb68a3a9401b3a0555393b7.jpg
WhatsApp-Image-2025-07-03-at-12_12_05-PM-4-_jpeg.rf.e80d78867042bc617987392a45f3dd13.jpg
WhatsApp-Image-2025-07-03-at-12_12_05-PM_jpeg.rf.238142b5cbbf0ae4860b13c896066b1a.jpg
WhatsApp-Image-2025-07-03-at-12_12_06-PM-3-_jpeg.rf.a6150e3bda1606340f744e495f073119.jpg
WhatsApp-Image-2025-07-03-at-12_12_06-PM-5-_jpeg.rf.ed1cf8f1c29954eef730b409ec0629ec.jpg
WhatsApp-Image-2025-07-03-at-12_12_07-PM-2-_jpeg.rf.80a14b46d11

In [ ]:
!ls /content/dataset/

data.yaml  README.dataset.txt  README.roboflow.txt  train


In [ ]:
with open("/content/dataset/data.yaml", "w") as f:
    f.write("""train: /content/dataset/train/images
val: /content/dataset/train/images

nc: 1
names: ['phone_number']
""")


In [ ]:
!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 100 \
  --data /content/dataset/data.yaml \
  --weights yolov5s.pt \
  --name tel_detect


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-08-05 11:00:11.374068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754391611.627554     979 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754391611.694187     979 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

In [ ]:
from google.colab import files
uploaded = files.upload()
image_path = next(iter(uploaded))

!python detect.py \
  --weights runs/train/tel_detect/weights/best.pt \
  --img 640 \
  --conf 0.25 \
  --source "{image_path}" \
  --project runs/detect \
  --name upload_test \
  --exist-ok
from IPython.display import Image, display
display(Image(f"runs/detect/upload_test/exp/{image_path}"))

Saving WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg to WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg
detect: weights=['runs/train/tel_detect/weights/best.pt'], source=WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=upload_test, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-423-g567c6646 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/yolov5/WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg: 640x384 1 phone_number, 42.1ms
Speed: 0.6ms pre-process, 42.1ms inference, 183.6ms 

FileNotFoundError: No such file or directory: 'runs/detect/upload_test/exp/WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg'

FileNotFoundError: No such file or directory: 'runs/detect/upload_test/exp/WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg'

<IPython.core.display.Image object>

In [ ]:
from google.colab import files
import os
uploaded = files.upload()
filename = next(iter(uploaded))
os.rename(filename, "input.jpg")

Saving WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg to WhatsApp Image 2025-07-03 at 12.12.07 PM (2) (1).jpeg


In [ ]:
!ls runs/train/


tel_detect


In [ ]:
!ls runs/train/tel_detect/weights/


best.pt  last.pt


In [ ]:
from PIL import Image
import pytesseract
import torch

model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/tel_detect/weights/best.pt', force_reload=True)


img = Image.open("input.jpg")


results = model(img)
boxes = results.xyxy[0]

for *box, conf, cls in boxes:
    x1, y1, x2, y2 = map(int, box)
    cropped = img.crop((x1, y1, x2, y2))

    text = pytesseract.image_to_string(cropped, config='--psm 7 -c tessedit_char_whitelist=0123456789')
    print("📞 Detected phone number:", text.strip())


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-8-5 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/co

📞 Detected phone number: 0 6 984 555 56


In [ ]:
from google.colab import files
import os
import zipfile
from PIL import Image
import pytesseract
import torch

print("📂 Please upload a ZIP file containing your 'img' folder")
uploaded = files.upload()

zip_filename = next(iter(uploaded))
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall()


os.remove(zip_filename)


model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/tel_detect/weights/best.pt', force_reload=True)


img_folder = 'img'
image_extensions = ['.jpg', '.jpeg', '.png']

for img_file in os.listdir(img_folder):
    if not any(img_file.lower().endswith(ext) for ext in image_extensions):
        continue

    img_path = os.path.join(img_folder, img_file)
    img = Image.open(img_path)


    results = model(img)
    boxes = results.xyxy[0]

    print(f"\n🔍 Processing {img_file}...")
    for i, (*box, conf, cls) in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        cropped = img.crop((x1, y1, x2, y2))

        text = pytesseract.image_to_string(cropped, config='--psm 7 -c tessedit_char_whitelist=0123456789')
        phone_number = text.strip()
        print(f"📞 Detection {i+1}: {phone_number}")


📂 Please upload a ZIP file containing your 'img' folder


Saving img (2).zip to img (2).zip


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-8-5 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):



🔍 Processing WhatsApp Image 2025-07-16 at 12.30.33 PM (1).jpeg...
📞 Detection 1: 0604217278

🔍 Processing WhatsApp Image 2025-07-16 at 12.31.36 PM (2).jpeg...
📞 Detection 1: 

🔍 Processing WhatsApp Image 2025-07-16 at 12.31.33 PM.jpeg...


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


📞 Detection 1: 0719498926

🔍 Processing WhatsApp Image 2025-07-16 at 12.31.28 PM (2).jpeg...
📞 Detection 1: 7

🔍 Processing WhatsApp Image 2025-07-03 at 12.13.39 PM (3).jpeg...


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


📞 Detection 1: 0608281669

🔍 Processing WhatsApp Image 2025-07-16 at 12.31.14 PM (6).jpeg...
📞 Detection 1: 273

🔍 Processing WhatsApp Image 2025-07-16 at 12.30.58 PM (3).jpeg...
📞 Detection 1: 0292


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [ ]:
import os
import re
import cv2
import torch
import easyocr
from PIL import Image
reader = easyocr.Reader(['fr', 'en'], gpu=True)
PHONE_RE = re.compile(r'0[67][0-9OIlL]{8}')

def clean_text(text):
    return text.upper().replace('O', '0').replace('I', '1').replace('L', '1')

def extract_phone_number_from_image(img):
    results = reader.readtext(img, detail=1, paragraph=False)
    candidates = []

    for result in results:
        _, text, conf = result
        cleaned = clean_text(text)
        digits = re.sub(r'[^0-9]', '', cleaned)

        if len(digits) == 10 and digits.startswith(('06', '07')):
            candidates.append((digits, conf))
        else:
            for match in PHONE_RE.findall(cleaned):
                num = re.sub(r'[^0-9]', '', match)
                if len(num) == 10:
                    candidates.append((num, conf))

    if candidates:
        best = max(candidates, key=lambda x: x[1])
        return best[0]
    else:
        return None

model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/tel_detect/weights/best.pt', force_reload=True)
IMG_DIR = 'img'
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

total = 0
detected = 0

for fname in sorted(os.listdir(IMG_DIR)):
    if not any(fname.lower().endswith(ext) for ext in image_extensions):
        continue

    total += 1
    path = os.path.join(IMG_DIR, fname)
    img_pil = Image.open(path)
    img_cv2 = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    results = model(img_pil)
    boxes = results.xyxy[0]

    found = False
    for i, (*box, conf, cls) in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        cropped = img_cv2[y1:y2, x1:x2]

        number = extract_phone_number_from_image(cropped)
        if number:
            print(f"{fname} → ✅ {number}")
            found = True
            detected += 1
            break

    if not found:
        print(f"{fname} → ❌ Aucun numéro détecté")

print(f"\n📊 Résultat final : {detected}/{total} images détectées ({(detected/total)*100:.2f}%)")

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-8-5 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


WhatsApp Image 2025-07-03 at 12.11.59 PM.jpeg → ❌ Aucun numéro détecté


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


WhatsApp Image 2025-07-16 at 12.31.59 PM (1).jpeg → ✅ 0605492644
WhatsApp Image 2025-07-16 at 12.32.00 PM (2).jpeg → ✅ 0710639249
WhatsApp Image 2025-07-16 at 12.32.00 PM (4).jpeg → ✅ 0714745663


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


WhatsApp Image 2025-07-16 at 12.32.00 PM.jpeg → ✅ 0627692001
WhatsApp Image 2025-07-16 at 12.32.01 PM (2).jpeg → ✅ 0716839831
WhatsApp Image 2025-07-16 at 12.32.01 PM (4).jpeg → ✅ 0711275805
WhatsApp Image 2025-07-16 at 12.32.01 PM.jpeg → ✅ 0699393152

📊 Résultat final : 123/164 images détectées (75.00%)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [ ]:
import os
import re
import cv2
import torch
import easyocr
import numpy as np
from PIL import Image

model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/tel_detect/weights/best.pt', force_reload=True)
model.conf = 0.25

reader = easyocr.Reader(['fr', 'en'], gpu=True)

IMG_DIR = 'img'
os.makedirs('missed', exist_ok=True)

PHONE_RE = re.compile(r'0[67][0-9OIlL]{8}')


def clean_text(text):
    return text.upper().replace('O', '0').replace('I', '1').replace('L', '1')


def enhance_crop_for_ocr(crop):
    gray = cv2.cvtColor(crop, cv2.COLOR_RGB2GRAY)
    gray = cv2.resize(gray, None, fx=2.5, fy=2.5, interpolation=cv2.INTER_CUBIC)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY, 31, 10)
    return cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)


def extract_phone_number_from_image(img):
    results = reader.readtext(img, detail=1, paragraph=False)
    candidates = []

    for result in results:
        _, text, conf = result
        cleaned = clean_text(text)
        digits = re.sub(r'[^0-9]', '', cleaned)

        if len(digits) == 10 and digits.startswith(('06', '07')):
            candidates.append((digits, conf))
        else:
            for match in PHONE_RE.findall(cleaned):
                num = re.sub(r'[^0-9]', '', match)
                if len(num) == 10:
                    candidates.append((num, conf))

    if candidates:
        return max(candidates, key=lambda x: x[1])[0]
    else:
        return None
total = 0
detected = 0

for fname in sorted(os.listdir(IMG_DIR)):
    if not fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        continue

    total += 1
    path = os.path.join(IMG_DIR, fname)
    img_pil = Image.open(path).convert("RGB")
    img_cv2 = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)

    results = model(img_pil)
    boxes = results.xyxy[0]

    img_h, img_w = img_rgb.shape[:2]
    numbers = []

    for (*box, conf, cls) in boxes:
        x1, y1, x2, y2 = map(int, box)

        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(img_w, x2), min(img_h, y2)

        if x2 <= x1 or y2 <= y1:
            continue

        cropped = img_rgb[y1:y2, x1:x2]
        if cropped.size == 0:
            continue

        enhanced_crop = enhance_crop_for_ocr(cropped)
        number = extract_phone_number_from_image(enhanced_crop)
        if number:
            numbers.append(number)
    if not numbers:
        fallback_number = extract_phone_number_from_image(img_rgb)
        if fallback_number:
            numbers.append(fallback_number)
            print(f"{fname} → ✅ (fallback) {fallback_number}")

    if numbers:
        detected += 1
        print(f"{fname} → ✅ {' | '.join(numbers)}")
    else:
        print(f"{fname} → ❌ Aucun numéro détecté")
        cv2.imwrite(os.path.join('missed', fname), img_cv2)

print(f"\n📊 Résultat final : {detected}/{total} images détectées ({(detected/total)*100:.2f}%)")

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-8-5 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


WhatsApp Image 2025-07-03 at 12.11.59 PM.jpeg → ❌ Aucun numéro détecté


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


WhatsApp Image 2025-07-03 at 12.12.00 PM (2).jpeg → ❌ Aucun numéro détecté


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


WhatsApp Image 2025-07-16 at 12.32.00 PM (4).jpeg → ✅ 0714745663
WhatsApp Image 2025-07-16 at 12.32.00 PM.jpeg → ✅ 0627692001


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


WhatsApp Image 2025-07-16 at 12.32.01 PM (2).jpeg → ❌ Aucun numéro détecté
WhatsApp Image 2025-07-16 at 12.32.01 PM (4).jpeg → ✅ 0711275805
WhatsApp Image 2025-07-16 at 12.32.01 PM.jpeg → ✅ 0699393152

📊 Résultat final : 126/164 images détectées (76.83%)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [ ]:
import os
import re
import cv2
import torch
import easyocr
import numpy as np
import pandas as pd
from PIL import Image

model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/tel_detect/weights/best.pt', force_reload=True)
model.conf = 0.25

reader = easyocr.Reader(['fr', 'en'], gpu=True)

IMG_DIR = 'img'
os.makedirs('missed', exist_ok=True)

PHONE_RE = re.compile(r'0[67][0-9OIlL]{8}')

def clean_text(text):
    return text.upper().replace('O', '0').replace('I', '1').replace('L', '1')

def enhance_crop_for_ocr(crop):
    gray = cv2.cvtColor(crop, cv2.COLOR_RGB2GRAY)
    gray = cv2.resize(gray, None, fx=2.5, fy=2.5, interpolation=cv2.INTER_CUBIC)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY, 31, 10)
    return cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)

def extract_numbers_from_image(img):
    results = reader.readtext(img, detail=1, paragraph=False)
    candidates = []

    for _, text, conf in results:
        cleaned = clean_text(text)
        digits = re.sub(r'[^0-9]', '', cleaned)

        if len(digits) == 10 and digits.startswith(('06', '07')):
            candidates.append((digits, conf))
        else:
            for match in PHONE_RE.findall(cleaned):
                num = re.sub(r'[^0-9]', '', match)
                if len(num) == 10:
                    candidates.append((num, conf))

    if candidates:
        return sorted(candidates, key=lambda x: x[1], reverse=True)
    else:
        return []

results_list = []
total = 0
detected = 0

for fname in sorted(os.listdir(IMG_DIR)):
    if not fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        continue

    total += 1
    path = os.path.join(IMG_DIR, fname)
    try:
        img_pil = Image.open(path).convert("RGB")
    except:
        print(f"❌ Failed to open: {fname}")
        continue

    img_cv2 = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
    img_h, img_w = img_rgb.shape[:2]

    numbers = []
    results = model(img_pil)
    boxes = results.xyxy[0]

    for (*box, conf, cls) in boxes:
        x1, y1, x2, y2 = map(int, box)
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(img_w, x2), min(img_h, y2)

        if x2 <= x1 or y2 <= y1:
            continue

        cropped = img_rgb[y1:y2, x1:x2]
        if cropped.size == 0:
            continue

        enhanced_crop = enhance_crop_for_ocr(cropped)
        crop_numbers = extract_numbers_from_image(enhanced_crop)
        numbers.extend(crop_numbers)

    fallback_numbers = extract_numbers_from_image(img_rgb)
    numbers.extend(fallback_numbers)

    seen = set()
    unique_numbers = []
    for num, conf in numbers:
        if num not in seen:
            seen.add(num)
            unique_numbers.append((num, conf))

    if unique_numbers:
        detected += 1
        formatted = " | ".join(f"{n} (conf={c:.2f})" for n, c in unique_numbers)
        print(f"{fname} → ✅ {formatted}")
        results_list.append({'image': fname, 'phone_numbers': formatted})
    else:
        print(f"{fname} → ❌ Aucun numéro détecté")
        cv2.imwrite(os.path.join('missed', fname), img_cv2)
        results_list.append({'image': fname, 'phone_numbers': 'None'})


df = pd.DataFrame(results_list)
df.to_csv('results_phone_numbers.csv', index=False)


print(f"\n📊 Résultat final : {detected}/{total} images détectées ({(detected/total)*100:.2f}%)")
print("📄 Résultats enregistrés dans 'results_phone_numbers.csv'")

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-8-5 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


WhatsApp Image 2025-07-03 at 12.11.59 PM.jpeg → ❌ Aucun numéro détecté


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


WhatsApp Image 2025-07-03 at 12.12.00 PM (2).jpeg → ❌ Aucun numéro détecté


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

WhatsApp Image 2025-07-16 at 12.32.01 PM (2).jpeg → ❌ Aucun numéro détecté


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


WhatsApp Image 2025-07-16 at 12.32.01 PM (4).jpeg → ✅ 0711275805 (conf=0.96)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


WhatsApp Image 2025-07-16 at 12.32.01 PM.jpeg → ✅ 0699393152 (conf=0.67)

📊 Résultat final : 126/164 images détectées (76.83%)
📄 Résultats enregistrés dans 'results_phone_numbers.csv'


In [ ]:
import os
import cv2
import pytesseract
import easyocr
import re
import pandas as pd
import numpy as np

# 🔧 Si vous êtes sur Windows, décommentez et adaptez cette ligne :
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# 📁 Dossier contenant les images
IMG_DIR = "img"
os.makedirs(IMG_DIR, exist_ok=True)  # crée le dossier si pas encore présent

# 🧠 Initialiser EasyOCR
reader = easyocr.Reader(['fr', 'en'], gpu=True)

# ✅ Regex pour numéros FR : 06xxxxxxxx ou 07xxxxxxxx
PHONE_RE = re.compile(r'0[67][0-9OIlL]{8}')

# Nettoyer les erreurs OCR communes
def clean_text(text):
    return text.upper().replace('O', '0').replace('I', '1').replace('L', '1')

# Extraire numéros valides depuis une liste de textes
def extract_phone_numbers(texts):
    numbers = []
    for text in texts:
        cleaned = clean_text(text)
        digits = re.sub(r'[^0-9]', '', cleaned)
        if len(digits) == 10 and digits.startswith(('06', '07')):
            numbers.append(digits)
        else:
            matches = PHONE_RE.findall(cleaned)
            for match in matches:
                num = re.sub(r'[^0-9]', '', match)
                if len(num) == 10:
                    numbers.append(num)
    return list(set(numbers))

# OCR avec EasyOCR
def extract_with_easyocr(image):
    results = reader.readtext(image)
    return [text for _, text, _ in results]

# OCR avec Tesseract
def extract_with_tesseract(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, None, fx=2.5, fy=2.5, interpolation=cv2.INTER_CUBIC)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY, 31, 10)
    text = pytesseract.image_to_string(thresh)
    return [line for line in text.split('\n') if line.strip() != '']

# 🔄 Lecture des images
results = []
for fname in sorted(os.listdir(IMG_DIR)):
    if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    print(f"📷 Traitement : {fname}")
    path = os.path.join(IMG_DIR, fname)
    img = cv2.imread(path)

    if img is None:
        print(f"❌ Impossible de lire l'image : {fname}")
        results.append({'image': fname, 'phone_numbers': 'Erreur de lecture'})
        continue

    # 1. Essayer EasyOCR
    texts = extract_with_easyocr(img)
    numbers = extract_phone_numbers(texts)

    # 2. Sinon fallback sur Tesseract
    if not numbers:
        texts = extract_with_tesseract(img)
        numbers = extract_phone_numbers(texts)

    # 3. Résultat
    if numbers:
        formatted = " | ".join(numbers)
        print(f"✅ Trouvé : {formatted}")
    else:
        formatted = "❌ Aucun numéro détecté"
        print(formatted)

    results.append({'image': fname, 'phone_numbers': formatted})

# 💾 Enregistrement CSV
df = pd.DataFrame(results)
df.to_csv('results_phone_numbers.csv', index=False)
print("\n📄 Résultats sauvegardés dans 'results_phone_numbers.csv'")


📷 Traitement : WhatsApp Image 2025-07-03 at 12.11.59 PM.jpeg
❌ Aucun numéro détecté
📷 Traitement : WhatsApp Image 2025-07-03 at 12.12.00 PM (2).jpeg
❌ Aucun numéro détecté
📷 Traitement : WhatsApp Image 2025-07-03 at 12.12.00 PM.jpeg
❌ Aucun numéro détecté
📷 Traitement : WhatsApp Image 2025-07-03 at 12.12.01 PM.jpeg
✅ Trouvé : 0707639242
📷 Traitement : WhatsApp Image 2025-07-03 at 12.12.05 PM (2).jpeg
✅ Trouvé : 0699347289
📷 Traitement : WhatsApp Image 2025-07-03 at 12.12.05 PM (4).jpeg
✅ Trouvé : 0726168484
📷 Traitement : WhatsApp Image 2025-07-03 at 12.12.05 PM.jpeg
✅ Trouvé : 0630424085
📷 Traitement : WhatsApp Image 2025-07-03 at 12.12.06 PM (3).jpeg
✅ Trouvé : 0601112531
📷 Traitement : WhatsApp Image 2025-07-03 at 12.12.06 PM (5).jpeg
✅ Trouvé : 0680469179
📷 Traitement : WhatsApp Image 2025-07-03 at 12.12.07 PM (2).jpeg
✅ Trouvé : 0698455356
📷 Traitement : WhatsApp Image 2025-07-03 at 12.12.07 PM (4).jpeg
✅ Trouvé : 0722244482
📷 Traitement : WhatsApp Image 2025-07-03 at 12.12.08 PM.

In [ ]:
pd.read_csv("results_phone_numbers.csv")


,image,phone_numbers
0,WhatsApp Image 2025-07-03 at 12.11.59 PM.jpeg,❌ Aucun numéro détecté
1,WhatsApp Image 2025-07-03 at 12.12.00 PM (2).jpeg,❌ Aucun numéro détecté
2,WhatsApp Image 2025-07-03 at 12.12.00 PM.jpeg,❌ Aucun numéro détecté
3,WhatsApp Image 2025-07-03 at 12.12.01 PM.jpeg,0707639242
4,WhatsApp Image 2025-07-03 at 12.12.05 PM (2).jpeg,0699347289
...,...,...
159,WhatsApp Image 2025-07-16 at 12.32.00 PM (4).jpeg,0714745663
160,WhatsApp Image 2025-07-16 at 12.32.00 PM.jpeg,0627692001
161,WhatsApp Image 2025-07-16 at 12.32.01 PM (2).jpeg,0774707700 | 0716839831
162,WhatsApp Image 2025-07-16 at 12.32.01 PM (4).jpeg,0711275805


In [ ]:
import pandas as pd

df = pd.read_csv("results_phone_numbers.csv")

detected = df[df['phone_numbers'] != "❌ Aucun numéro détecté"]

num_detected_images = detected.shape[0]

print(f"Number of images with at least one phone number detected: {num_detected_images}")

Number of images with at least one phone number detected: 128


In [ ]:
import os

import cv2
import pytesseract
import easyocr
import re
import pandas as pd
import numpy as np
from imutils.perspective import four_point_transform

# Optional: If you're on Windows
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

IMG_DIR = "img"
os.makedirs(IMG_DIR, exist_ok=True)

reader = easyocr.Reader(['fr', 'en'], gpu=True)

# Phone number regex: accepts 06/07 and handles OCR confusion
PHONE_RE = re.compile(r'0[67][0-9OIlL]{8}')

def clean_text(text):
    return text.upper().replace('O', '0').replace('I', '1').replace('L', '1')

def extract_phone_numbers(texts):
    numbers = []
    for text in texts:
        cleaned = clean_text(text)
        digits = re.sub(r'[^0-9]', '', cleaned)
        if len(digits) == 10 and digits.startswith(('06', '07')):
            numbers.append(digits)
        else:
            matches = PHONE_RE.findall(cleaned)
            for match in matches:
                num = re.sub(r'[^0-9]', '', match)
                if len(num) == 10:
                    numbers.append(num)
    return list(set(numbers))

def rotate_image(image, angle):
    return cv2.rotate(image, {
        90: cv2.ROTATE_90_CLOCKWISE,
        180: cv2.ROTATE_180,
        270: cv2.ROTATE_90_COUNTERCLOCKWISE
    }[angle])

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    denoise = cv2.fastNlMeansDenoising(gray, h=30)
    resized = cv2.resize(denoise, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    blur = cv2.GaussianBlur(resized, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 35, 11)
    return thresh

def extract_with_easyocr(image):
    results = reader.readtext(image)
    return [text for _, text, confidence in results if confidence > 0.3]

def extract_with_tesseract(image):
    processed = preprocess_image(image)
    raw_text = pytesseract.image_to_string(processed)
    return [line for line in raw_text.split("\n") if line.strip() != '']

def try_all_orientations(image, extractor_func):
    all_texts = []
    for angle in [0, 90, 180, 270]:
        rotated = rotate_image(image, angle) if angle != 0 else image
        try:
            texts = extractor_func(rotated)
            all_texts.extend(texts)
        except Exception as e:
            continue
    return all_texts

# Process all images
results = []

for fname in sorted(os.listdir(IMG_DIR)):
    if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    print(f"📷 Processing: {fname}")
    path = os.path.join(IMG_DIR, fname)
    img = cv2.imread(path)

    if img is None:
        results.append({'image': fname, 'phone_numbers': '❌ Cannot read image'})
        continue

    # Try EasyOCR
    texts_easy = try_all_orientations(img, extract_with_easyocr)
    numbers = extract_phone_numbers(texts_easy)

    # Fallback on Tesseract
    if not numbers:
        texts_tess = try_all_orientations(img, extract_with_tesseract)
        numbers = extract_phone_numbers(texts_tess)

    # Save results
    if numbers:
        result_str = " | ".join(numbers)
        print(f"✅ Found: {result_str}")
    else:
        result_str = "❌ No number detected"
        print(result_str)

    results.append({'image': fname, 'phone_numbers': result_str})

# Save as CSV
df = pd.DataFrame(results)
df.to_csv('results_phone_numbers.csv', index=False)
print("\n📄 Results saved to 'results_phone_numbers.csv'")


ModuleNotFoundError: No module named 'pytesseract'

In [ ]:
!pip install pytesseract
!sudo apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!pip install easyocr
!pip install opencv-python
!pip install torch torchvision torchaudio
!pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 18.8 MB/s eta 0:00:00


In [ ]:
import os
import re
import cv2
import torch
import easyocr
import numpy as np
import pandas as pd
from PIL import Image

# 🔧 Chargement du modèle YOLOv5 personnalisé
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/tel_detect2/weights/best.pt', force_reload=True)
model.conf = 0.25

# 🔤 OCR avec EasyOCR
reader = easyocr.Reader(['fr', 'en'], gpu=True)

# 📂 Dossier des images
IMG_DIR = 'img'
os.makedirs('missed', exist_ok=True)

# 📞 Expression régulière pour les numéros FR
PHONE_RE = re.compile(r'0[67][0-9OIlL]{8}')

# 🔠 Nettoyage du texte OCR
def clean_text(text):
    return text.upper().replace('O', '0').replace('I', '1').replace('L', '1')

# 🎨 Amélioration du crop pour OCR
def enhance_crop_for_ocr(crop):
    gray = cv2.cvtColor(crop, cv2.COLOR_RGB2GRAY)
    gray = cv2.resize(gray, None, fx=2.5, fy=2.5, interpolation=cv2.INTER_CUBIC)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY, 31, 10)
    return cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)

# 🔄 Correction d’orientation
def deskew_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    coords = np.column_stack(np.where(gray < 255))
    if coords.size == 0:
        return image
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
    return cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

# 🔍 Extraction des numéros à partir d'une image
def extract_numbers_from_image(img):
    results = reader.readtext(img, detail=1, paragraph=False)
    candidates = []
    for _, text, conf in results:
        cleaned = clean_text(text)
        digits = re.sub(r'[^0-9]', '', cleaned)
        if len(digits) == 10 and digits.startswith(('06', '07')):
            candidates.append((digits, conf))
        else:
            for match in PHONE_RE.findall(cleaned):
                num = re.sub(r'[^0-9]', '', match)
                if len(num) == 10:
                    candidates.append((num, conf))
    return sorted(candidates, key=lambda x: x[1], reverse=True) if candidates else []

# ⚫ Test version normale et inversée pour OCR
def try_both_contrasts(img):
    normal = extract_numbers_from_image(img)
    inverted = extract_numbers_from_image(255 - img)
    return max(normal, inverted, key=lambda x: x[0][1] if x else 0) if (normal or inverted) else []

# 🧠 Détection manuelle des zones texte si YOLO échoue
def fallback_regions(img_rgb):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    _, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(255 - binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    crops = []
    for cnt in contours:
        x, y, w_box, h_box = cv2.boundingRect(cnt)
        if w_box > 30 and h_box > 15:
            crop = img_rgb[y:y + h_box, x:x + w_box]
            crops.append(crop)
    return crops

# 🔁 Traitement des images
results_list = []
total = 0
detected = 0

for fname in sorted(os.listdir(IMG_DIR)):
    if not fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        continue

    total += 1
    path = os.path.join(IMG_DIR, fname)
    try:
        img_pil = Image.open(path).convert("RGB")
    except:
        print(f"❌ Failed to open: {fname}")
        continue

    img_cv2 = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
    img_h, img_w = img_rgb.shape[:2]

    numbers = []
    results = model(img_pil)
    boxes = results.xyxy[0]

    if boxes.shape[0]:
        for (*box, conf, cls) in boxes:
            x1, y1, x2, y2 = map(int, box)
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(img_w, x2), min(img_h, y2)
            if x2 <= x1 or y2 <= y1:
                continue
            cropped = img_rgb[y1:y2, x1:x2]
            if cropped.size == 0:
                continue
            deskewed = deskew_image(cropped)
            enhanced = enhance_crop_for_ocr(deskewed)
            crop_numbers = try_both_contrasts(enhanced)
            numbers.extend(crop_numbers)
    else:
        for crop in fallback_regions(img_rgb):
            deskewed = deskew_image(crop)
            enhanced = enhance_crop_for_ocr(deskewed)
            crop_numbers = try_both_contrasts(enhanced)
            numbers.extend(crop_numbers)

    seen = set()
    unique_numbers = []
    for num, conf in numbers:
        if num not in seen:
            seen.add(num)
            unique_numbers.append((num, conf))

    if unique_numbers:
        detected += 1
        formatted = " | ".join(f"{n} (conf={c:.2f})" for n, c in unique_numbers)
        print(f"{fname} → ✅ {formatted}")
        results_list.append({'image': fname, 'phone_numbers': formatted})
    else:
        print(f"{fname} → ❌ Aucun numéro détecté")
        cv2.imwrite(os.path.join('missed', fname), img_cv2)
        results_list.append({'image': fname, 'phone_numbers': 'None'})

# 💾 Enregistrement des résultats
df = pd.DataFrame(results_list)
df.to_csv('results_phone_numbers.csv', index=False)

print(f"\n📊 Résultat final : {detected}/{total} images détectées ({(detected/total)*100:.2f}%)")
print("📄 Résultats enregistrés dans 'results_phone_numbers.csv'")

ModuleNotFoundError: No module named 'easyocr'